# Setup

In [2]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_csv("/kaggle/input/toxic-message-classifier-dataset/train_cleaned.csv")

# Actual Model

## Creating the encoder

In [4]:
NUM_ROWS = 150000
BATCH_SIZE = 64

MAX_TOKENS = 5000
MAX_LENGTH = 200
encoder = tf.keras.layers.TextVectorization(max_tokens=MAX_TOKENS, output_sequence_length=MAX_LENGTH)
encoder.adapt(data.head(NUM_ROWS)["comment_text"].tolist())

vocab = np.array(encoder.get_vocabulary())
vocab[:20]

len(vocab)

5000

## Creating the model

In [7]:
# Sets random seed so results are identical every time
SEED = 1
tf.random.set_seed(SEED)
np.random.seed(SEED)
tf.keras.utils.set_random_seed(SEED)

# TOXIC_CATEGORIES = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
TOXIC_CATEGORIES = ["toxic"]

model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=512,
        mask_zero=True
    ),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(TOXIC_CATEGORIES), activation="sigmoid")
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
             optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=["accuracy"])


### Training the model

In [8]:
# Classification for all types of toxicity
multiDf = data.head(NUM_ROWS)[["comment_text"] + TOXIC_CATEGORIES]

x = multiDf["comment_text"]
y = multiDf[TOXIC_CATEGORIES]

splitter = StratifiedShuffleSplit(random_state=1, test_size=0.2)

for train,test in splitter.split(x, y[TOXIC_CATEGORIES[0]]):
    training_data = x.iloc[train]
    training_target = y.iloc[train]
    validation_data = x.iloc[test]
    validation_target = y.iloc[test]

In [9]:
# Early Stopping
callback = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=2)

history = model.fit(training_data, training_target, epochs=10, validation_data=(validation_data, validation_target), callbacks=[callback], batch_size=32)

Epoch 1/10
3734/3734 [==============================] - 176s 42ms/step - loss: 0.1603 - accuracy: 0.9482 - val_loss: 0.1148 - val_accuracy: 0.9598
Epoch 2/10
3734/3734 [==============================] - 153s 41ms/step - loss: 0.0949 - accuracy: 0.9647 - val_loss: 0.1181 - val_accuracy: 0.9599
Epoch 4/10
3734/3734 [==============================] - 153s 41ms/step - loss: 0.0852 - accuracy: 0.9676 - val_loss: 0.1303 - val_accuracy: 0.9583


### Getting test data

In [10]:
# Classification for all toxicity values
test_data = pd.read_csv("/kaggle/input/toxic-message-classifier-dataset/test.csv")
test_labels = pd.read_csv("/kaggle/input/toxic-message-classifier-dataset/test_labels.csv")

test_labels = test_labels
merged_df = test_labels.merge(test_data, left_on="id", right_on="id")
merged_df = merged_df.loc[(merged_df["toxic"] != -1) & (merged_df["severe_toxic"] != -1) & (merged_df["obscene"] != -1) & (merged_df["threat"] != -1) & (merged_df["insult"] != -1) & (merged_df["identity_hate"] != -1)]

In [11]:
# Tests all rows with a value of 0 or 1 for all toxicity values

test_df = merged_df["comment_text"]
test_target = merged_df[TOXIC_CATEGORIES]
model.evaluate(test_df, test_target)

2000/2000 [==============================] - 37s 18ms/step - loss: 0.2250 - accuracy: 0.9085


[0.22501592338085175, 0.908515453338623]

In [12]:
# Test where any toxicity value is 1

test_df = merged_df["comment_text"]
query = " | ".join([f"({label} == 1)" for label in TOXIC_CATEGORIES])
filtered_df = merged_df.query(query)

filtered_test_dataset = filtered_df["comment_text"]
filtered_df_target = filtered_df[TOXIC_CATEGORIES]
model.evaluate(filtered_test_dataset, filtered_df_target)

191/191 [==============================] - 3s 15ms/step - loss: 0.4688 - accuracy: 0.8289


[0.468755841255188, 0.8288998603820801]

In [13]:
# Test where all toxicity values are 0

test_df = merged_df["comment_text"]
query = " | ".join([f"({label} == 0)" for label in TOXIC_CATEGORIES])
filtered_df = merged_df.query(query)

filtered_test_dataset = filtered_df["comment_text"]
filtered_df_target = filtered_df[TOXIC_CATEGORIES]
model.evaluate(filtered_test_dataset, filtered_df_target)

1809/1809 [==============================] - 34s 19ms/step - loss: 0.1994 - accuracy: 0.9169


[0.1993737518787384, 0.9168912172317505]

In [20]:
submission_set = pd.read_csv('/kaggle/input/toxic-message-classifier-dataset/test.csv')
submission_set.head()

x_test = submission_set['comment_text'].values
y_testing = model.predict(x_test, verbose=1, batch_size=BATCH_SIZE)

y_testing

2394/2394 [==============================] - 51s 21ms/step


array([[9.72522914e-01, 1.66898251e-01, 8.58725786e-01, 4.40095291e-02,
        7.41641939e-01, 1.35891005e-01],
       [9.66314599e-03, 1.64660469e-05, 9.05902183e-04, 2.81564753e-05,
        9.14453412e-04, 1.15424096e-04],
       [1.11537397e-01, 6.67511660e-04, 1.61998477e-02, 1.36845326e-03,
        1.82596836e-02, 2.68361974e-03],
       ...,
       [1.05915396e-02, 2.06470977e-05, 1.02800620e-03, 3.51235976e-05,
        1.01354544e-03, 1.32377070e-04],
       [1.45154878e-01, 6.24501321e-04, 1.62635241e-02, 1.21272530e-03,
        2.14106832e-02, 2.76782596e-03],
       [9.54435945e-01, 1.05474807e-01, 7.85146058e-01, 5.32792062e-02,
        6.61822379e-01, 1.50208518e-01]], dtype=float32)

In [24]:
submission_df = pd.DataFrame(columns = ['id','toxic','severe_toxic','obscene','threat','insult','identity_hate'])

submission_df['id'] = submission_set['id']
submission_df['toxic'] = [0 if x[0] < 0.5 else 1 for x in y_testing]
submission_df['severe_toxic'] = [0 if x[1] < 0.5 else 1 for x in y_testing]
submission_df['obscene'] = [0 if x[2] < 0.5 else 1 for x in y_testing]
submission_df['threat'] = [0 if x[3] < 0.5 else 1 for x in y_testing]
submission_df['insult'] = [0 if x[4] < 0.5 else 1 for x in y_testing]
submission_df['identity_hate'] = [0 if x[5] < 0.5 else 1 for x in y_testing]

submission_df.head()

submission_df.to_csv('/kaggle/working/submission.csv', index=False)

In [40]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_3 (TextV  (None, 200)              0         
 ectorization)                                                   
                                                                 
 embedding_7 (Embedding)     (None, 200, 512)          2560000   
                                                                 
 bidirectional_7 (Bidirectio  (None, 200, 128)         295424    
 nal)                                                            
                                                                 
 bidirectional_8 (Bidirectio  (None, 64)               31104     
 nal)                                                            
                                                                 
 dense_14 (Dense)            (None, 64)                4160      
                                                      